In [ ]:
import numpy as np
%reload_ext autoreload
%autoreload 2


In [206]:
val = torch.tensor(np.array([1,0,0,0,0,0,0,0,0,0])).repeat(1003)

In [207]:
unique, counts = np.unique(val, return_counts=True)
dict(zip(unique, counts))

{0: 9027, 1: 1003}

In [208]:
n_samples = 500

In [221]:
from torch.utils.data import TensorDataset
from torch.nn.functional import softmax
from estimators.data.dataloaders import *
sampler = ImbalancedDatasetSampler(ds, n_samples=1000)
# sampler.weights = torch.ones(len(val)) / 2
print(sampler.weights)
ds = TensorDataset(val)
dl = DataLoader(ds, batch_size=128, sampler=sampler)

{1: 1003, 0: 9027}
tensor([0.0010, 0.0001, 0.0001,  ..., 0.0001, 0.0001, 0.0001],
       dtype=torch.float64)


In [222]:
for batch in dl:
    v = batch[0].numpy()
#     v = ds[torch.multinomial(weights,100, replacement=True)].numpy()
    unique, counts = np.unique(v, return_counts=True)
    print(counts)

[66 62]
[67 61]
[69 59]
[59 69]
[58 70]
[64 64]
[65 63]
[67 61]
[69 59]
[63 65]
[64 64]
[69 59]
[69 59]
[64 64]
[62 66]
[52 76]
[62 66]
[63 65]
[68 60]
[62 66]
[71 57]
[60 68]
[57 71]
[67 61]
[61 67]
[70 58]
[54 74]
[70 58]
[74 54]
[63 65]
[63 65]
[69 59]
[62 66]
[66 62]
[69 59]
[67 61]
[73 55]
[69 59]
[69 59]
[60 68]
[67 61]
[66 62]
[68 60]
[70 58]
[67 61]
[65 63]
[59 69]
[67 61]
[67 61]
[54 74]
[62 66]
[61 67]
[54 74]
[68 60]
[62 66]
[66 62]
[74 54]
[69 59]
[56 72]
[70 58]
[59 69]
[57 71]
[73 55]
[65 63]
[68 60]
[54 74]
[72 56]
[60 68]
[60 68]
[70 58]
[69 59]
[64 64]
[68 60]
[57 71]
[59 69]
[61 67]
[82 46]
[60 68]
[22 24]


In [139]:
import torch
from torch.nn.functional import softmax
# probs = torch.tensor(n_samples / counts)

# probs = torch.tensor(np.ones(n_samples)/counts)
softmax(weights, dim=0)

tensor([0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010, 0.0010, 

In [156]:
for batch in dl:
    v = batch[0].numpy()
#     v = ds[torch.multinomial(weights,100, replacement=True)].numpy()
    unique, counts = np.unique(v, return_counts=True)
    print(counts)

[86 14]
[90 10]
[91  9]
[90 10]
[93  7]
[93  7]
[84 16]
[88 12]
[90 10]
[87 13]


In [103]:
counts / n_samples

array([0.9, 0.1])

In [ ]:
from utilities.postprocessing.handlers.functional import *
from utilities.visualisation import *

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import rosbag_pandas
%matplotlib notebook
import matplotlib.pyplot as plt
# path = '/media/francesco/saetta/krock-dataset/new-test-random/bags/querry-big-10-10-1558293330.0385795.bag'
from utilities.postprocessing.handlers.functional import *
import glob
from utilities.visualisation import *
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
paths = glob.glob('/media/francesco/saetta/krock-dataset/new-test-random/csvs_patches/*.csv')
# hm = read_image('./maps/test/querry-big-10.png')
# df = rosbag_pandas.bag_to_dataframe(paths[0])
# df = parse_dataframe(df) 
# df = add_hm_coordinates2df(df, hm)
# df = clean_dataframe(df, hm, lower_bound=1)
# df = add_advancement(df, 100)

def vis_dataset(path):
    fig = plt.figure()
    hm = read_image('./maps/test/querry-big-10.png')
# #     df = rosbag_pandas.bag_to_dataframe(path)
#     df = parse_dataframe(df) 
#     df = add_hm_coordinates2df(df, hm)
#     df = clean_dataframe(df, hm, lower_bound=1)
#     df = add_advancement(df, 100)

    vis = PatchesAndDataframeVisualization(pd.read_csv(path), hm=hm)
    vis.show_patches_on_the_map()
    
    return df
    
interact(vis_dataset, path=paths)